# Find a Vaccine - Get a Text to Your Phone!

In [1]:
import requests
import pandas as pd
import time
from datetime import datetime
import os
from twilio.rest import Client
from IPython.display import clear_output

In [3]:
def convert_time(string):
    date = datetime.strptime(string, '%Y-%m-%dT%H:%M:%S.%f%z')
    date = datetime.strftime(date, '%A %B %d, %Y, at %H:%M %p')
    return date

In [4]:
def successful():
    message = """
    Appointments found at the following location:
    {}, {} NC {}
    Access via this link: {}
    Checking again in {} minutes.
    """.format(usable_data['address'], usable_data['city'], usable_data['postal_code'], usable_data['url'], success_sleep_mins)
    client.api.account.messages.create(
    to=outphone,
    from_=fromphone,
    body=message)

In [5]:
def failure(send_failure_text = True):
    if send_failure_text:
        current_time = datetime.now()
        message = """
        No available appointments found. Checking for new appointments every {} minutes.
        """.format(failure_sleep_mins)
        client.api.account.messages.create(
        to=outphone,
        from_ =fromphone,
        body=message)

## Twilio Auth

In [ ]:
account_sid = '' # hide
auth_token = '' # hide

outphone = '' # hide
fromphone = '' #hide

client = Client(account_sid, auth_token)

## Parameters

In [6]:
empty_list = [] # Create empty list to compare against for appointments. No need to change this.

In [7]:
city_list = (('CHARLOTTE', 'ASHEVILLE', 'HARRISBURG', 'CONCORD', 'ROCKINGHAM')) # specify which cities you'd like to check

In [8]:
view_times = False # view_times = True will print appointment times to console if found

In [9]:
success_sleep_mins = 30 # How many min to wait after finding appts
failure_sleep_mins= 2 # How many min to wait after not finding appts

success_sleep_secs = success_sleep_mins * 60
failure_sleep_secs = failure_sleep_mins * 60

## Main Loop

In [ ]:
while True:
    response = requests.get('https://www.vaccinespotter.org/api/v0/states/NC.json')
    if response.status_code != 200:
        raise Exception('Status code is not 200!')
    data = response.json()['features']
    for i in data:
        usable_data = i['properties']
        if usable_data['city'].upper() in city_list and \
        usable_data['appointments_available_all_doses'] == True \
        and usable_data['appointments'] != empty_list:
            clear_output()
            print('Appointments available at the following locations:')
            print('{}, {} NC {}'.format(usable_data['address'], usable_data['city'], usable_data['postal_code']))
            successful()
            if view_times:
                for x in usable_data['appointments']:
                    print('Time: {}'.format(convert_time(x['time'])))
            time.sleep(success_sleep_secs)
            clear_output()
            print('Getting new data!')
    else:
        failure(True)
        time.sleep(failure_sleep_secs)

Appointments available at the following locations:
1500 E BROAD AVE, ROCKINGHAM NC 28379
